# Basic SQLAlchemy SQL Expressions

In [81]:
import urllib
from os import environ

from sqlalchemy import create_engine, func, desc, cast, JSON
from sqlalchemy import type_coerce
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.sql import union, select, and_, or_, not_, text
from sqlalchemy.sql import bindparam
from sqlalchemy.sql.functions import coalesce

# Setup

In [131]:
driver = environ.get('SQL_DRIVER', '{ODBC Driver 17 for SQL Server}')
host = environ.get('SQL_HOST', 'sql-fabulous')
db = environ.get('SQL_DB', 'ScratchDB')
user = environ.get('SQL_USER', 'sa')
pw = environ.get('SQL_PASSWORD', 'HelloWorld1')

con_str = f'DRIVER={driver};SERVER={host};DATABASE={db};UID={user};PWD={pw}'

params = urllib.parse.quote_plus(con_str)  

# 'echo' emits generated sql
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", echo=True)

# Define Table Schema

In [132]:
metadata = MetaData()

users = Table('users', metadata,
            Column('id', Integer, autoincrement=False, primary_key=True),
            Column('name', String(50), nullable=False),
            Column('fullname', String(255), nullable=False),
        )

addresses = Table('addresses', metadata,
                Column('id', Integer, autoincrement=False, primary_key=True),
                Column('user_id', None, ForeignKey('users.id')),
                Column('email_address', String(255), nullable=False)
            )

metadata.create_all(engine, checkfirst=True)

2019-08-24 17:40:59,714 INFO sqlalchemy.engine.base.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2019-08-24 17:40:59,715 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 17:40:59,721 INFO sqlalchemy.engine.base.Engine SELECT schema_name()
2019-08-24 17:40:59,721 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 17:40:59,794 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-08-24 17:40:59,795 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 17:40:59,797 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS NVARCHAR(60)) AS anon_1
2019-08-24 17:40:59,798 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 17:40:59,805 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[C

In [133]:
# Establish connection
conn = engine.connect()

# Inserts

In [5]:
ins = users.insert().values(id=1, name='jack', fullname='Jack Jones')
conn.execute(ins)

2019-08-24 15:18:34,466 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2019-08-24 15:18:34,470 INFO sqlalchemy.engine.base.Engine (1, 'jack', 'Jack Jones')
2019-08-24 15:18:34,475 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2019-08-24 15:18:34,664 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2019-08-24 15:18:34,665 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2019-08-24 15:18:34,668 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
# Multi-insert
# Can also update(), delete()
conn.execute(addresses.insert(), [
        {'id': 1, 'user_id': 1, 'email_address' : 'jack@yahoo.com'},
        {'id': 2, 'user_id': 1, 'email_address' : 'jack@msn.com'},
        {'id': 3, 'user_id': 2, 'email_address' : 'www@www.org'},
        {'id': 4, 'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

2019-08-24 15:18:34,850 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (id, user_id, email_address) VALUES (?, ?, ?)
2019-08-24 15:18:34,851 INFO sqlalchemy.engine.base.Engine ((1, 1, 'jack@yahoo.com'), (2, 1, 'jack@msn.com'), (3, 2, 'www@www.org'), (4, 2, 'wendy@aol.com'))
2019-08-24 15:18:34,859 INFO sqlalchemy.engine.base.Engine COMMIT


# Select

### Make sure to close result cursors as below.

In [8]:
s = select([users])
result = conn.execute(s) # returns cursor

for r in result:
    print(r)
    print(f"name: {r['name']}; fullname: {r['fullname']}")
    print(f"name: {r[1]}; fullname: {r[2]}")
    print(f"name: {r[users.c.name]}, fullname:{r[users.c.fullname]}")
          
result.close()

2019-08-24 15:18:34,996 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-08-24 15:18:34,998 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones')
name: jack; fullname: Jack Jones
name: jack; fullname: Jack Jones
name: jack, fullname:Jack Jones
(2, 'wendy', 'Wendy Williams')
name: wendy; fullname: Wendy Williams
name: wendy; fullname: Wendy Williams
name: wendy, fullname:Wendy Williams


In [9]:
# Explicit columns
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)

for row in result:
    print(row)
    
result.close()

2019-08-24 15:18:35,170 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2019-08-24 15:18:35,174 INFO sqlalchemy.engine.base.Engine ()
('jack', 'Jack Jones')
('wendy', 'Wendy Williams')


In [10]:
# Cross join
result = conn.execute(select([users, addresses]))
for row in result:
    print(row)
    
result.close()

2019-08-24 15:18:35,283 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses
2019-08-24 15:18:35,284 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')
(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')
(2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com')
(2, 'wendy', 'Wendy Williams', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [11]:
# Inner join
result = conn.execute(
            select([users, addresses])\
            .where(users.c.id == addresses.c.user_id)
        )
for row in result:
    print(row)
    
result.close()

2019-08-24 15:18:35,416 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-08-24 15:18:35,417 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


# Operators

In [12]:
# "== None" becomes "IS / IS NOT NULL"


# Conjunctions

In [13]:
stmt = select([(users.c.fullname + ", " + addresses.c.email_address).label('title')])\
        .where(and_(
            users.c.name.like('j%'),
            users.c.id == addresses.c.user_id,
            or_(
                addresses.c.email_address == 'wendy@aol.com',
                addresses.c.email_address == 'jack@yahoo.com'
            ),
            not_(users.c.id > 5)
        ))

result = conn.execute(stmt)

for r in result:
    print(r)
    
result.close()

2019-08-24 15:18:35,773 INFO sqlalchemy.engine.base.Engine SELECT users.fullname + ? + addresses.email_address AS title 
FROM users, addresses 
WHERE users.name LIKE ? AND users.id = addresses.user_id AND (addresses.email_address = ? OR addresses.email_address = ?) AND users.id <= ?
2019-08-24 15:18:35,774 INFO sqlalchemy.engine.base.Engine (', ', 'j%', 'wendy@aol.com', 'jack@yahoo.com', 5)
('Jack Jones, jack@yahoo.com',)


In [14]:
# Equivalent using multiple where clauses (method chaining)
stmt = select([(users.c.fullname + ", " + addresses.c.email_address).label('title')])\
        .where(users.c.name.like('j%'))\
        .where(users.c.id == addresses.c.user_id)\
        .where(or_(
                addresses.c.email_address == 'wendy@aol.com',
                addresses.c.email_address == 'jack@yahoo.com'
            ))\
        .where(not_(users.c.id > 5))

result = conn.execute(stmt)

for r in result:
    print(r)
    
result.close()

2019-08-24 15:18:36,006 INFO sqlalchemy.engine.base.Engine SELECT users.fullname + ? + addresses.email_address AS title 
FROM users, addresses 
WHERE users.name LIKE ? AND users.id = addresses.user_id AND (addresses.email_address = ? OR addresses.email_address = ?) AND users.id <= ?
2019-08-24 15:18:36,007 INFO sqlalchemy.engine.base.Engine (', ', 'j%', 'wendy@aol.com', 'jack@yahoo.com', 5)
('Jack Jones, jack@yahoo.com',)


# Explicit SQL with Parameters

In [15]:
# Explicit T-SQL
s = text(
    "SELECT users.fullname + ', ' + addresses.email_address AS title "
    "FROM users, addresses "
    "WHERE users.id = addresses.user_id "
        "AND users.name BETWEEN :x AND :y "
        "AND (addresses.email_address LIKE :e1 "
                "OR addresses.email_address LIKE :e2)")
# Or: s2 = s.bindparams(x='m', y='z', e1='%@aol.com', e2='%@msn.com')
r = conn.execute(s, x='m', y='z', e1='%@aol.com', e2='%@msn.com')

for row in r:
    print(row)
r.close()

2019-08-24 15:18:36,116 INFO sqlalchemy.engine.base.Engine SELECT users.fullname + ', ' + addresses.email_address AS title FROM users, addresses WHERE users.id = addresses.user_id AND users.name BETWEEN ? AND ? AND (addresses.email_address LIKE ? OR addresses.email_address LIKE ?)
2019-08-24 15:18:36,118 INFO sqlalchemy.engine.base.Engine ('m', 'z', '%@aol.com', '%@msn.com')
('Wendy Williams, wendy@aol.com',)


# Group By / Order By

In [16]:
stmt = select([addresses.c.user_id,
                func.count(addresses.c.id).label('num_addresses')]).\
        group_by(addresses.c.user_id)\
        .order_by(addresses.c.user_id, desc("num_addresses"))

conn.execute(stmt).fetchall()

2019-08-24 15:18:36,258 INFO sqlalchemy.engine.base.Engine SELECT addresses.user_id, count(addresses.id) AS num_addresses 
FROM addresses GROUP BY addresses.user_id ORDER BY addresses.user_id, num_addresses DESC
2019-08-24 15:18:36,259 INFO sqlalchemy.engine.base.Engine ()


[(1, 2), (2, 2)]

# Table Aliasing / Sub-Queries

In [17]:
a1 = addresses.alias('a1')
a2 = addresses.alias('a2')
s = select([users]).\
    where(and_(
            users.c.id == a1.c.user_id,
            users.c.id == a2.c.user_id,
            a1.c.email_address == 'jack@msn.com',
            a2.c.email_address == 'jack@yahoo.com'
    ))

conn.execute(s).fetchall()

2019-08-24 15:18:36,370 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users, addresses AS a1, addresses AS a2 
WHERE users.id = a1.user_id AND users.id = a2.user_id AND a1.email_address = ? AND a2.email_address = ?
2019-08-24 15:18:36,371 INFO sqlalchemy.engine.base.Engine ('jack@msn.com', 'jack@yahoo.com')


[(1, 'jack', 'Jack Jones')]

In [18]:
addresses_subq = s.alias()
s = select([users.c.name])\
    .where(users.c.id == addresses_subq.c.id)
conn.execute(s).fetchall()

2019-08-24 15:18:36,491 INFO sqlalchemy.engine.base.Engine SELECT users.name 
FROM users, (SELECT users.id AS id, users.name AS name, users.fullname AS fullname 
FROM users, addresses AS a1, addresses AS a2 
WHERE users.id = a1.user_id AND users.id = a2.user_id AND a1.email_address = ? AND a2.email_address = ?) AS anon_1 
WHERE users.id = anon_1.id
2019-08-24 15:18:36,492 INFO sqlalchemy.engine.base.Engine ('jack@msn.com', 'jack@yahoo.com')


[('jack',)]

# Joins

In [19]:
# Can also use outerjoin()
s = select([users.c.fullname])\
        .select_from(
            users.join(addresses)
    )
conn.execute(s).fetchall()

2019-08-24 15:18:36,613 INFO sqlalchemy.engine.base.Engine SELECT users.fullname 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-08-24 15:18:36,615 INFO sqlalchemy.engine.base.Engine ()


[('Jack Jones',), ('Jack Jones',), ('Wendy Williams',), ('Wendy Williams',)]

In [20]:
s = select([users, addresses.c.email_address])\
        .select_from(
            users.join(addresses,
                addresses.c.email_address.like(users.c.name + '%'))
    )
conn.execute(s).fetchall()

2019-08-24 15:18:36,735 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.email_address 
FROM users JOIN addresses ON addresses.email_address LIKE users.name + ?
2019-08-24 15:18:36,737 INFO sqlalchemy.engine.base.Engine ('%',)


[(1, 'jack', 'Jack Jones', 'jack@yahoo.com'),
 (1, 'jack', 'Jack Jones', 'jack@msn.com'),
 (2, 'wendy', 'Wendy Williams', 'wendy@aol.com')]

# Functions

In [21]:
print(func.now())
print(func.current_timestamp())

now()
CURRENT_TIMESTAMP


In [22]:
conn.execute(
    select([
        func.max(addresses.c.email_address, type_=String).
            label('maxemail')
        ])
).scalar()

2019-08-24 15:18:37,000 INFO sqlalchemy.engine.base.Engine SELECT max(addresses.email_address) AS maxemail 
FROM addresses
2019-08-24 15:18:37,002 INFO sqlalchemy.engine.base.Engine ()


'www@www.org'

# Window Functions

In [23]:
s = select([
    users.c.id,
    func.row_number().over(order_by=users.c.name)
])

conn.execute(s).fetchall()

2019-08-24 15:18:37,124 INFO sqlalchemy.engine.base.Engine SELECT users.id, row_number() OVER (ORDER BY users.name) AS anon_1 
FROM users
2019-08-24 15:18:37,125 INFO sqlalchemy.engine.base.Engine ()


[(1, 1), (2, 2)]

In [24]:
s = select([
        users.c.id,
        func.count(addresses.c.id).over(
            partition_by=users.c.name,
            order_by=users.c.id,
            rows=(None, None)
        )
    ])\
    .select_from(
        users.join(addresses))
    
conn.execute(s).fetchall()

2019-08-24 15:18:37,257 INFO sqlalchemy.engine.base.Engine SELECT users.id, count(addresses.id) OVER (PARTITION BY users.name ORDER BY users.id ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS anon_1 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-08-24 15:18:37,260 INFO sqlalchemy.engine.base.Engine ()


[(1, 2), (1, 2), (2, 2), (2, 2)]

# Casts

In [26]:
s = select([cast(users.c.id, String)])
conn.execute(s).fetchall()

2019-08-24 15:20:09,344 INFO sqlalchemy.engine.base.Engine SELECT CAST(users.id AS VARCHAR(max)) AS anon_1 
FROM users
2019-08-24 15:20:09,345 INFO sqlalchemy.engine.base.Engine ()


[('1',), ('2',)]

In [31]:
# Coerces but does not cast.
d = type_coerce({'some_key': {'foo': 'bar'}}, JSON)['some_key']
d

# Set Ops

In [33]:
# Also, union_all(), intersect(), except_()
u = union(
    addresses.select()
        .where(addresses.c.email_address == 'foo@bar.com'),
    addresses.select()
        .where(addresses.c.email_address.like('%@yahoo.com')),
).order_by(addresses.c.email_address)

conn.execute(u).fetchall()

2019-08-24 15:27:24,648 INFO sqlalchemy.engine.base.Engine SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address = ? UNION SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ? ORDER BY addresses.email_address
2019-08-24 15:27:24,648 INFO sqlalchemy.engine.base.Engine ('foo@bar.com', '%@yahoo.com')


[(1, 1, 'jack@yahoo.com')]

# Scalar Selects

In [38]:
stmt = select([func.count(addresses.c.id)])\
        .where(users.c.id == addresses.c.user_id)\
        .as_scalar()

# This creates a correlated sub-query
conn.execute(select([users.c.name, stmt])).fetchall()

# This creates a named correlated sub-query
stmt = select([func.count(addresses.c.id)])\
        .where(users.c.id == addresses.c.user_id)\
        .label("address_count")

# This creates a correlated sub-query
conn.execute(select([users.c.name, stmt])).fetchall()

2019-08-24 15:34:38,201 INFO sqlalchemy.engine.base.Engine SELECT users.name, (SELECT count(addresses.id) AS count_1 
FROM addresses 
WHERE users.id = addresses.user_id) AS anon_1 
FROM users
2019-08-24 15:34:38,204 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 15:34:38,208 INFO sqlalchemy.engine.base.Engine SELECT users.name, (SELECT count(addresses.id) AS count_1 
FROM addresses 
WHERE users.id = addresses.user_id) AS address_count 
FROM users
2019-08-24 15:34:38,209 INFO sqlalchemy.engine.base.Engine ()


[('jack', 2), ('wendy', 2)]

# Correlated sub-queries

In [39]:
stmt = select([addresses.c.user_id])\
        .where(addresses.c.user_id == users.c.id)\
        .where(addresses.c.email_address == 'jack@yahoo.com')

# SqlAlchemy interprets the var as a sub-query
enclosing_stmt = select([users.c.name]).where(users.c.id == stmt)

conn.execute(enclosing_stmt).fetchall()

2019-08-24 15:37:39,381 INFO sqlalchemy.engine.base.Engine SELECT users.name 
FROM users 
WHERE users.id = (SELECT addresses.user_id 
FROM addresses 
WHERE addresses.user_id = users.id AND addresses.email_address = ?)
2019-08-24 15:37:39,382 INFO sqlalchemy.engine.base.Engine ('jack@yahoo.com',)


[('jack',)]

In [40]:
# Specifying a specific correlated table when more than 1 is involved.
# Also, correlate_except()
stmt = select([users.c.id])\
        .where(users.c.id == addresses.c.user_id)\
        .where(users.c.name == 'jack')\
        .correlate(addresses)

enclosing_stmt = select([users.c.name, addresses.c.email_address])\
        .select_from(users.join(addresses))\
        .where(users.c.id == stmt)

conn.execute(enclosing_stmt).fetchall()

2019-08-24 15:41:53,057 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE users.id = (SELECT users.id 
FROM users 
WHERE users.id = addresses.user_id AND users.name = ?)
2019-08-24 15:41:53,059 INFO sqlalchemy.engine.base.Engine ('jack',)


[('jack', 'jack@yahoo.com'), ('jack', 'jack@msn.com')]

In [45]:
# Disable correlation - creates non-correlated sub-query
stmt = select([users.c.id])\
        .where(users.c.name == 'wendy')\
        .correlate(None)

enclosing_stmt = select([users.c.name])\
        .where(users.c.id == stmt)

conn.execute(enclosing_stmt).fetchall()

2019-08-24 15:45:44,409 INFO sqlalchemy.engine.base.Engine SELECT users.name 
FROM users 
WHERE users.id = (SELECT users.id 
FROM users 
WHERE users.name = ?)
2019-08-24 15:45:44,410 INFO sqlalchemy.engine.base.Engine ('wendy',)


[('wendy',)]

# Group By, Order By, Offset Fetch

In [47]:
stmt = select([users.c.name]).order_by(users.c.name.desc())
conn.execute(stmt).fetchall()

2019-08-24 15:55:33,584 INFO sqlalchemy.engine.base.Engine SELECT users.name 
FROM users ORDER BY users.name DESC
2019-08-24 15:55:33,585 INFO sqlalchemy.engine.base.Engine ()


[('wendy',), ('jack',)]

In [49]:
stmt = select([users.c.name, func.count(addresses.c.id)])\
        .select_from(users.join(addresses))\
        .group_by(users.c.name)\
        .having(func.length(users.c.name) > 4)

conn.execute(stmt).fetchall()

2019-08-24 15:57:07,090 INFO sqlalchemy.engine.base.Engine SELECT users.name, count(addresses.id) AS count_1 
FROM users JOIN addresses ON users.id = addresses.user_id GROUP BY users.name 
HAVING LEN(users.name) > ?
2019-08-24 15:57:07,091 INFO sqlalchemy.engine.base.Engine (4,)


[('wendy', 2)]

In [54]:
stmt = select([users.c.name])\
        .select_from(users.join(addresses))\
        .distinct()

conn.execute(stmt).fetchall()

2019-08-24 15:59:56,889 INFO sqlalchemy.engine.base.Engine SELECT DISTINCT users.name 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-08-24 15:59:56,890 INFO sqlalchemy.engine.base.Engine ()


[('jack',), ('wendy',)]

In [57]:
stmt = select([users.c.name, addresses.c.email_address])\
            .select_from(users.join(addresses))\
            .order_by(users.c.name.asc())\
            .limit(1).offset(1)

conn.execute(stmt).fetchall()

2019-08-24 16:01:20,531 INFO sqlalchemy.engine.base.Engine SELECT anon_1.name, anon_1.email_address 
FROM (SELECT users.name AS name, addresses.email_address AS email_address, ROW_NUMBER() OVER (ORDER BY users.name ASC) AS mssql_rn 
FROM users JOIN addresses ON users.id = addresses.user_id) AS anon_1 
WHERE mssql_rn > ? AND mssql_rn <= ? + ?
2019-08-24 16:01:20,532 INFO sqlalchemy.engine.base.Engine (1, 1, 1)


[('jack', 'jack@msn.com')]

# Inserts, Updates, Deletes

In [59]:
stmt = users.update()\
        .values(fullname="Fullname: " + users.c.name)

conn.execute(stmt)

2019-08-24 16:03:52,568 INFO sqlalchemy.engine.base.Engine UPDATE users SET fullname=(? + users.name)
2019-08-24 16:03:52,570 INFO sqlalchemy.engine.base.Engine ('Fullname: ',)
2019-08-24 16:03:52,573 INFO sqlalchemy.engine.base.Engine COMMIT


In [66]:
stmt = users.insert()\
        .values([
    {'id':7, 'name':'name1', 'fullname':'name1'},
    {'id':8, 'name':'name2', 'fullname':'name2'},
    {'id':9, 'name':'name3', 'fullname':'name3'},
])

conn.execute(stmt)

2019-08-24 16:10:05,094 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?)
2019-08-24 16:10:05,095 INFO sqlalchemy.engine.base.Engine (7, 'name1', 'name1', 8, 'name2', 'name2', 9, 'name3', 'name3')
2019-08-24 16:10:05,102 INFO sqlalchemy.engine.base.Engine COMMIT


In [63]:
# Override a parameter
stmt = users.insert()\
        .values(name=bindparam('_name') + " .. name")

conn.execute(stmt, [
    {'id':4, '_name':'name1', 'fullname':'name1'},
    {'id':5, '_name':'name2', 'fullname':'name2'},
    {'id':6, '_name':'name3', 'fullname':'name3'},
])

2019-08-24 16:07:57,510 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, (? + ?), ?)
2019-08-24 16:07:57,511 INFO sqlalchemy.engine.base.Engine ((4, 'name1', ' .. name', 'name1'), (5, 'name2', ' .. name', 'name2'), (6, 'name3', ' .. name', 'name3'))
2019-08-24 16:07:57,514 INFO sqlalchemy.engine.base.Engine COMMIT


In [68]:
stmt = users.update()\
        .where(users.c.name == 'jack')\
        .values(name='ed')

conn.execute(stmt)

2019-08-24 16:11:14,935 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.name = ?
2019-08-24 16:11:14,936 INFO sqlalchemy.engine.base.Engine ('ed', 'jack')
2019-08-24 16:11:14,974 INFO sqlalchemy.engine.base.Engine COMMIT


In [69]:
stmt = users.update()\
        .where(users.c.name == bindparam('oldname'))\
        .values(name=bindparam('newname'))

conn.execute(stmt, [
    {'oldname':'jack', 'newname':'jack2'},
    {'oldname':'wendy', 'newname':'wendy2'},
    {'oldname':'jim', 'newname':'jim2'},
])

2019-08-24 16:14:36,712 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.name = ?
2019-08-24 16:14:36,713 INFO sqlalchemy.engine.base.Engine (('jack2', 'jack'), ('wendy2', 'wendy'), ('jim2', 'jim'))
2019-08-24 16:14:36,722 INFO sqlalchemy.engine.base.Engine COMMIT


In [113]:
# Correlated update
stmt = users.update().\
        values(fullname="Name test2: "+addresses.c.email_address)\
        .where(users.c.id == addresses.c.id)\
        .where(addresses.c.email_address.endswith('.com'))

conn.execute(stmt)

2019-08-24 16:51:49,320 INFO sqlalchemy.engine.base.Engine UPDATE users SET fullname=(? + addresses.email_address) FROM users, addresses WHERE users.id = addresses.id AND (addresses.email_address LIKE '%' + ?)
2019-08-24 16:51:49,323 INFO sqlalchemy.engine.base.Engine ('Name test2: ', '.com')
2019-08-24 16:51:49,331 INFO sqlalchemy.engine.base.Engine COMMIT


In [120]:
# Correlated update with explicit dictionary syntax
stmt = users.update().\
        values({
            users.c.name: "Name: " + addresses.c.email_address,
            users.c.fullname: "Email: " + addresses.c.email_address
        })\
        .where(users.c.id == addresses.c.id)\
        .where(addresses.c.email_address.endswith('.com'))

conn.execute(stmt)

2019-08-24 17:13:38,323 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=(? + addresses.email_address), fullname=(? + addresses.email_address) FROM users, addresses WHERE users.id = addresses.id AND (addresses.email_address LIKE '%' + ?)
2019-08-24 17:13:38,324 INFO sqlalchemy.engine.base.Engine ('Name: ', 'Email: ', '.com')
2019-08-24 17:13:38,329 INFO sqlalchemy.engine.base.Engine COMMIT


In [121]:
conn.execute(users.delete().where(users.c.fullname.startswith('jack')))

2019-08-24 17:18:51,947 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.fullname LIKE ? + '%'
2019-08-24 17:18:51,948 INFO sqlalchemy.engine.base.Engine ('jack',)
2019-08-24 17:18:52,121 INFO sqlalchemy.engine.base.Engine COMMIT


In [130]:
# Subquery delete
stmt = users.delete()\
        .where(users.c.id.notin_(
            select([addresses.c.user_id])
        ))\
        .where(users.c.fullname.startswith('ed'))

conn.execute(stmt)

2019-08-24 17:28:19,642 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id NOT IN (SELECT addresses.user_id 
FROM addresses) AND (users.fullname LIKE ? + '%')
2019-08-24 17:28:19,647 INFO sqlalchemy.engine.base.Engine ('ed',)
2019-08-24 17:28:19,661 INFO sqlalchemy.engine.base.Engine COMMIT
